In [1]:
import editdistance
import re
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
#Считывание данных КроссЛексиси
f=open('PARON1.txt', "r", encoding="cp1251")
words = list(f.readlines())
f.close()

In [3]:
#Делает из изначального представления обычное написание
def word(string):
    string = re.sub(r"[^а-яА-Я]", "", string)
    return string.lower()

In [4]:
def clean(string):
    string = re.sub(r"[а-яА-Я][-][а-яА-Я]", ",", string)
    string = re.sub(r"[^а-яА-Я,]", "", string)
    return string

In [5]:
"""
Определение части речи
1-сущ
2-глагол
3-прилагательное
4-наречие
"""
def pos(string):
    return int(string[2])

"""
для существительных-число
1-единственный
0-множественный
"""
def num(string):
    return string[4]
    
    
"""
для существительных-род
1-мужской
2-женский
3-средний
4-общий
"""
def gender(string):
    if pos(string)==1:
        return int(string[6])
    else:
        return None

In [6]:
#корень
def root(string):
    result = re.findall(r'[+][а-яА-Я1-9]+[-*\n]', string)
    return clean(str(result))


#окончание
def ending(string):
    result = re.findall(r'[*][а-яА-Я]+[-\n]', string)
    return clean(str(result))

#возвратная частица
def particle(string):
    result = re.findall(r'[-](СЯ)[\n]', string)
    return clean(str(result))

#приставка right
def preffix(string):
    result = re.findall(r' [а-яА-Я-]+[+]', string)
    if result!=[]:
        result = re.sub(r"[-+]", " ", result[0]).split()
#    result = re.sub(r"[+ ]", "", result)
    return result#clean(str(result))

#суффикс right
def suffix(string):
    result = re.findall(r'[+][а-яА-Я-]+[*]', string)
    if result!=[]:
        result = re.findall(r'[-][а-яА-Я-]+[*]', result[0])
        if result!=[]:
            result = re.sub(r"[-*]", " ", result[0]).split()
    return result#clean(str(result))



In [7]:
#расстояние Левенштейна в морфах и по буквам в корне
def pre(word1,word2):
    return editdistance.eval(preffix(word1),preffix(word2))
def post(word1,word2):
    return editdistance.eval(suffix(word1),suffix(word2))
def rootdist(word1,word2):
    return editdistance.eval(root(word1),root(word2))

In [8]:
def compare(w1,w2):
    return [pre(w1,w2),post(w1,w2),rootdist(w1,w2)]

# word2vec

In [332]:
#word2vec
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('araneum_1_600_2.bin', binary=True) #~helenahuddy/CNN/ruscorpora.model.bin
#def get_word_variants(word):
#    return word_vectors[word]   #if 'word' in word_vectors.vocab
#word_vectors.similarity('абажур','абажурчик')

In [334]:
#Чтобы выглядели, как в модели
def pluspos(aword):
    if pos(aword)==1:
        return word(aword)+'_NOUN'
    elif pos(aword)==2:
        return word(aword)+'_VERB'
    elif pos(aword)==3:
        return word(aword)+'_ADJ'
    elif pos(aword)==4:
        return word(aword)+'_ADV'

In [336]:
def similar(w1,w2):
    return word_vectors.similarity(pluspos(w1),pluspos(w2))

# Степень покрытия словаря Красных

In [9]:
#Считываем словарь
file=open('CrossLexica/RED.txt', "r", encoding="cp1251")
krasnih = list(file.readlines())
file.close()

In [521]:
krasnih

['+ 1 1 1 +АБОН-ЕМ-ЕНТ*\n',
 '- 1 1 1 +АБОН-ЕНТ*\n',
 '+ 3 0 0 +АБОН-ЕМ-ЕНТ-Н*ЫЙ\n',
 '- 3 0 0 +АБОН-ЕНТ-Н*ЫЙ\n',
 '- 3 0 0 +АБОН-ЕНТ-СК*ИЙ\n',
 '+ 3 0 0 +АБСТРАКТ-Н*ЫЙ\n',
 '- 3 0 0 +АБСТРАКЦ-ИОН-ИСТ-СК*ИЙ\n',
 '+ 1 1 1 +АВАНГАРД*\n',
 '- 1 1 1 +АВАНГАРД-ИЗМ*\n',
 '+ 3 0 0 +АВАНГАРД-ИСТ-СК*ИЙ\n',
 '- 3 0 0 +АВАНГАРД-Н*ЫЙ\n',
 '+ 1 1 2 +АВАНТЮР*А\n',
 '- 1 1 1 +АВАНТЮР-ИЗМ*\n',
 '+ 3 0 0 +АВАНТЮР-ИСТ-ИЧ-ЕСК*ИЙ\n',
 '- 3 0 0 +АВАНТЮР-ИСТ-ИЧ-Н*ЫЙ\n',
 '- 3 0 0 +АВАНТЮР-Н*ЫЙ\n',
 '+ 3 0 0 +АВТОБИОГРАФ-ИЧ-ЕСК*ИЙ\n',
 '- 3 0 0 +АВТОБИОГРАФ-ИЧ-Н*ЫЙ\n',
 '+ 1 1 2 +АВТОМАТ-ИЗ-АЦИ*Я\n',
 '- 1 1 1 +АВТОМАТ-ИЗМ*\n',
 '- 1 1 2 +АВТОМАТ-ИК*А\n',
 '+ 3 0 0 +АВТОМАТ-ИЗ-ИР-ОВ-АНН*ЫЙ\n',
 '- 3 0 0 +АВТОМАТ-ИЧ-ЕСК*ИЙ\n',
 '- 3 0 0 +АВТОМАТ-ИЧ-Н*ЫЙ\n',
 '+ 3 0 0 +АВТОР-ИТ-АР-Н*ЫЙ\n',
 '- 3 0 0 +АВТОР-ИТ-ЕТ-Н*ЫЙ\n',
 '+ 1 1 3 +АГЕНТ-СТВ*О\n',
 '- 1 1 2 +АГЕНТ-УР*А\n',
 '+ 3 0 0 +АДАПТ-АЦИ-ОНН*ЫЙ\n',
 '- 3 0 0 +АДАПТ-ИВ-Н*ЫЙ\n',
 '- 3 0 0 +АДАПТ-ИР-ОВ-АНН*ЫЙ\n',
 '+ 3 0 0 -АД+МИНИСТР-АТ-ИВ-Н*ЫЙ\n',
 '- 3 0

In [10]:
#Разбиваем по частям речи
nouns_krasnih=[]
for i in range (len(krasnih)):
    if pos(krasnih[i])==1:
        nouns_krasnih.append(krasnih[i])
        
        
verbs_krasnih=[]
for i in range (len(krasnih)):
    if pos(krasnih[i])==2:
        verbs_krasnih.append(krasnih[i])
        
        
adv_krasnih=[]
for i in range (len(krasnih)):
    if pos(krasnih[i])==3:
        adv_krasnih.append(krasnih[i])
        

In [11]:
#От начального представляения словаря переходим к представлению в виде пар паронимов
def vocab_krasn(words):
    paronyms=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                paronyms.append([word(words[i]),word(words[j])])
                
    return paronyms

In [344]:
vocab_krasn(krasnih)

[['абонемент', 'абонент'],
 ['абонементный', 'абонентный'],
 ['абонементный', 'абонентский'],
 ['абонентный', 'абонентский'],
 ['абстрактный', 'абстракционистский'],
 ['авангард', 'авангардизм'],
 ['авангардистский', 'авангардный'],
 ['авантюра', 'авантюризм'],
 ['авантюристический', 'авантюристичный'],
 ['авантюристический', 'авантюрный'],
 ['авантюристичный', 'авантюрный'],
 ['автобиографический', 'автобиографичный'],
 ['автоматизация', 'автоматизм'],
 ['автоматизация', 'автоматика'],
 ['автоматизм', 'автоматика'],
 ['автоматизированный', 'автоматический'],
 ['автоматизированный', 'автоматичный'],
 ['автоматический', 'автоматичный'],
 ['авторитарный', 'авторитетный'],
 ['агентство', 'агентура'],
 ['адаптационный', 'адаптивный'],
 ['адаптационный', 'адаптированный'],
 ['адаптивный', 'адаптированный'],
 ['административный', 'администраторский'],
 ['адресант', 'адресат'],
 ['аккредитация', 'аккредитив'],
 ['акселерат', 'акселератор'],
 ['акселерат', 'акселерация'],
 ['акселератор', 'акс

In [12]:
def cross (list1,etalon):
    count=0
    for i in etalon:
        if i in list1:
            count+=1
    return [count/(len(etalon)),count/(len(list1))]

# Статистика

In [13]:
#Представление в виде пар, но слова в изначальной разбитой на морфемы форме
def vocab_krasn_pure(words):
    paronyms=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                paronyms.append([words[i],words[j]])
                
    return paronyms



In [14]:
#Подсчёт префиксов
def count_pref(words):
    r=[]
    for i in words:
        for j in preffix(i):
            r.append(j)
    counter = Counter()  
    for k in r:
        counter[k]+=1
    return dict(counter),sum(counter.values())

In [15]:
#Подсчёт суффиксов
def count_suff(words):
    r=[]
    for i in words:
        for j in suffix(i):
            r.append(j)
    counter = Counter()  
    for k in r:
        counter[k]+=1
    return dict(counter),sum(counter.values())

In [16]:
#Подсчёт расстояний Левенштейна
def count(words):
    r=[]
    group=vocab_krasn_pure(words)
    for i in group:
        r.append(compare(i[0],i[1]))
    r=[str(i).strip('[]') for i in r]
    counter = Counter()  
    for k in r:
        counter[k]+=1
    return dict(counter)

In [17]:
#Подсчёт рода
def count_gender(words):
    r=[]
    group=vocab_krasn_pure(words)
    for i in group:
        r.append(gender(i[0])==gender(i[1]))
    counter = Counter()  
    for k in r:
        counter[k]+=1
    return dict(counter)

In [1087]:
sorted(count_pref(nouns)[0].values(),reverse=True)[0:3]

[444, 374, 340]

In [1090]:
for i in parts_of_speech_krasnih+parts_of_speech:
    print('Самые частые преффиксы',sorted(count_pref(i)[0].items(), key=l, reverse=True)[0:3],count_pref(i)[1])
    print('Самые частые суффиксы',sorted(count_suff(i)[0].items(), key=l, reverse=True)[0:3],count_suff(i)[1])
    print('Средняя частота преффиксов',count_pref(i)[1]/len(i))
    print('Средняя частота суффиксов',count_suff(i)[1]/len(i))
    print('общее количество разных морфем,преф', len(sorted(count_pref(i)[0].items(), key=l, reverse=True)))
    print('общее количество разных морфем,суфф', len(sorted(count_suff(i)[0].items(), key=l, reverse=True)))
    array1=sorted(count_pref(i)[0].values(),reverse=True)[0:3]
    array2=[100*k/count_pref(i)[1] for k in array1]
    print('Частота самых частых преф',array2)
    array3=sorted(count_suff(i)[0].values(),reverse=True)[0:3]
    array4=[100*k/count_suff(i)[1] for k in array3]
    print('Частота самых частых суф',array2)
    print('\n')

Самые частые преффиксы [('О', 25), ('ПРО', 23), ('У', 22)] 279
Самые частые суффиксы [('ОСТ', 97), ('К', 88), ('Н', 64)] 958
Средняя частота преффиксов 0.3290094339622642
Средняя частота суффиксов 1.1297169811320755
общее количество разных морфем,преф 35
общее количество разных морфем,суфф 123
Частота самых частых преф [8.960573476702509, 8.243727598566307, 7.885304659498208]
Частота самых частых суф [8.960573476702509, 8.243727598566307, 7.885304659498208]


Самые частые преффиксы [('О', 39), ('ПО', 27), ('ОТ', 27)] 342
Самые частые суффиксы [('Н', 15), ('ОВ', 11), ('ОМ', 4)] 46
Средняя частота преффиксов 0.9941860465116279
Средняя частота суффиксов 0.13372093023255813
общее количество разных морфем,преф 30
общее количество разных морфем,суфф 12
Частота самых частых преф [11.403508771929825, 7.894736842105263, 7.894736842105263]
Частота самых частых суф [11.403508771929825, 7.894736842105263, 7.894736842105263]


Самые частые преффиксы [('НЕ', 69), ('О', 42), ('С', 31)] 422
Самые част

In [932]:
#Статистика для (не)совпаений родов у имён существительных словаря Красных
sorted(count_gender(nouns_krasnih).items(), key=l, reverse=True)

[(True, 340), (False, 250)]

In [752]:
#Статистика для существительных (расст в преффиксах/аффиксах/корнях)
l = lambda x: x[1]
sorted(count(nouns_krasnih).items(), key=l, reverse=True) 


[('0, 1, 0', 181),
 ('0, 2, 0', 142),
 ('1, 0, 0', 76),
 ('0, 3, 0', 40),
 ('0, 1, 1', 37),
 ('0, 2, 1', 30),
 ('0, 1, 2', 9),
 ('0, 1, 3', 9),
 ('0, 1, 4', 7),
 ('0, 0, 1', 7),
 ('0, 4, 0', 7),
 ('0, 0, 0', 5),
 ('1, 1, 0', 5),
 ('0, 2, 2', 4),
 ('1, 0, 3', 4),
 ('0, 1, 8', 3),
 ('1, 0, 4', 3),
 ('1, 0, 1', 3),
 ('0, 2, 5', 3),
 ('0, 2, 4', 2),
 ('1, 1, 1', 2),
 ('0, 0, 2', 2),
 ('0, 3, 4', 1),
 ('0, 1, 7', 1),
 ('0, 2, 3', 1),
 ('0, 3, 1', 1),
 ('0, 2, 6', 1),
 ('1, 1, 2', 1),
 ('0, 0, 8', 1),
 ('1, 2, 1', 1),
 ('0, 0, 3', 1)]

In [755]:
sorted(count(verbs_krasnih).items(), key=l, reverse=True) 

[('1, 0, 0', 324),
 ('0, 0, 1', 4),
 ('0, 1, 0', 3),
 ('0, 2, 0', 3),
 ('0, 0, 0', 3),
 ('1, 0, 2', 2),
 ('1, 0, 1', 2),
 ('0, 3, 0', 1),
 ('2, 0, 0', 1),
 ('0, 0, 2', 1)]

In [756]:
sorted(count(adv_krasnih).items(), key=l, reverse=True) 

[('0, 1, 0', 390),
 ('0, 2, 0', 277),
 ('0, 1, 1', 92),
 ('0, 3, 0', 49),
 ('0, 2, 1', 48),
 ('1, 0, 0', 24),
 ('0, 1, 2', 18),
 ('1, 1, 1', 12),
 ('0, 3, 1', 11),
 ('0, 4, 0', 11),
 ('1, 1, 0', 9),
 ('1, 2, 0', 8),
 ('0, 0, 0', 7),
 ('0, 2, 2', 6),
 ('0, 0, 2', 4),
 ('2, 0, 0', 3),
 ('0, 5, 0', 2),
 ('0, 0, 3', 1),
 ('0, 4, 1', 1),
 ('1, 0, 1', 1),
 ('0, 0, 1', 1),
 ('0, 1, 3', 1),
 ('0, 2, 3', 1)]

In [839]:
sorted(count(nouns).items(), key=l, reverse=True) 



[('1, 1, 0', 22043),
 ('1, 0, 0', 17033),
 ('1, 2, 0', 13091),
 ('1, 1, 1', 12839),
 ('0, 1, 0', 9856),
 ('1, 2, 1', 8356),
 ('0, 0, 0', 7047),
 ('0, 2, 0', 6589),
 ('0, 1, 1', 2920),
 ('0, 3, 0', 2160),
 ('1, 3, 0', 2032),
 ('0, 2, 1', 1947),
 ('1, 1, 2', 1890),
 ('2, 1, 0', 1581),
 ('1, 2, 2', 1463),
 ('1, 3, 1', 1448),
 ('1, 0, 1', 1050),
 ('2, 2, 0', 1027),
 ('2, 1, 1', 940),
 ('2, 2, 1', 749),
 ('2, 0, 0', 568),
 ('0, 4, 0', 536),
 ('0, 1, 2', 472),
 ('0, 0, 1', 451),
 ('0, 3, 1', 418),
 ('1, 4, 0', 402),
 ('1, 1, 3', 352),
 ('1, 0, 2', 319),
 ('0, 2, 2', 223),
 ('2, 3, 0', 165),
 ('0, 0, 2', 132),
 ('1, 4, 1', 131),
 ('1, 3, 2', 124),
 ('2, 1, 2', 121),
 ('0, 1, 3', 115),
 ('2, 4, 0', 105),
 ('2, 3, 1', 102),
 ('1, 2, 3', 100),
 ('2, 2, 2', 93),
 ('2, 0, 1', 74),
 ('0, 2, 3', 59),
 ('2, 4, 1', 52),
 ('2, 1, 3', 51),
 ('0, 4, 1', 38),
 ('0, 3, 2', 31),
 ('2, 0, 2', 28),
 ('1, 3, 3', 23),
 ('0, 0, 3', 19),
 ('1, 4, 2', 16),
 ('0, 5, 0', 15),
 ('2, 5, 0', 11),
 ('2, 2, 3', 11),
 ('2

In [840]:
sorted(count(verbs).items(), key=l, reverse=True) 



[('1, 0, 0', 43271),
 ('1, 0, 1', 16450),
 ('1, 1, 0', 11311),
 ('1, 1, 1', 8182),
 ('0, 0, 0', 3425),
 ('2, 0, 0', 2093),
 ('1, 0, 2', 2090),
 ('1, 1, 2', 1840),
 ('0, 0, 1', 1603),
 ('2, 0, 1', 1119),
 ('0, 1, 0', 1044),
 ('0, 1, 1', 720),
 ('2, 1, 0', 622),
 ('2, 1, 1', 389),
 ('1, 2, 1', 303),
 ('2, 0, 2', 294),
 ('0, 0, 2', 278),
 ('2, 1, 2', 268),
 ('1, 0, 3', 227),
 ('1, 2, 2', 180),
 ('0, 1, 2', 156),
 ('1, 2, 0', 85),
 ('0, 2, 0', 47),
 ('0, 0, 3', 42),
 ('1, 1, 3', 36),
 ('1, 3, 0', 30),
 ('0, 2, 1', 28),
 ('2, 0, 3', 26),
 ('2, 2, 2', 18),
 ('0, 3, 0', 14),
 ('2, 2, 1', 12),
 ('2, 3, 0', 9),
 ('1, 3, 1', 8),
 ('0, 2, 2', 6),
 ('2, 3, 1', 5),
 ('0, 1, 3', 5),
 ('2, 2, 0', 4),
 ('2, 1, 3', 2)]

In [769]:
sorted(count(adv).items(), key=l, reverse=True) 



[('1, 1, 0', 18445),
 ('1, 1, 1', 14995),
 ('1, 0, 0', 11961),
 ('1, 2, 0', 10345),
 ('1, 2, 1', 6666),
 ('0, 1, 0', 4006),
 ('0, 2, 0', 3372),
 ('1, 1, 2', 3137),
 ('0, 1, 1', 1908),
 ('1, 2, 2', 1396),
 ('2, 1, 0', 1395),
 ('2, 1, 1', 1320),
 ('0, 0, 0', 1070),
 ('1, 3, 0', 1019),
 ('0, 2, 1', 1008),
 ('0, 3, 0', 826),
 ('2, 2, 0', 805),
 ('1, 1, 3', 549),
 ('2, 0, 0', 516),
 ('0, 1, 2', 388),
 ('1, 3, 1', 380),
 ('2, 2, 1', 367),
 ('1, 0, 1', 365),
 ('1, 3, 2', 293),
 ('2, 1, 2', 277),
 ('1, 2, 3', 238),
 ('0, 2, 2', 221),
 ('0, 4, 0', 197),
 ('0, 3, 1', 170),
 ('2, 2, 2', 166),
 ('1, 0, 2', 140),
 ('2, 1, 3', 108),
 ('1, 4, 0', 105),
 ('3, 1, 0', 62),
 ('0, 1, 3', 59),
 ('1, 4, 1', 50),
 ('0, 0, 1', 48),
 ('3, 1, 1', 44),
 ('2, 2, 3', 40),
 ('0, 0, 2', 38),
 ('2, 3, 0', 37),
 ('1, 0, 3', 37),
 ('0, 5, 0', 34),
 ('0, 2, 3', 32),
 ('0, 4, 1', 31),
 ('1, 5, 0', 30),
 ('3, 0, 0', 27),
 ('3, 2, 0', 23),
 ('2, 3, 1', 19),
 ('2, 3, 2', 19),
 ('2, 0, 2', 18),
 ('2, 0, 1', 18),
 ('0, 3, 2',

# Аффиксальный критерий

In [18]:
nouns=[]
verbs=[]
adv=[]
adj=[]
for i in words:
    if pos(i)==1:
        nouns.append(i)
    elif pos(i)==2:
        verbs.append(i)
    elif pos(i)==3:
        adv.append(i)
    elif pos(i)==4:
        adj.append(i)

In [1071]:
prefixes=[0,1,2]
suffixes=[0,1,2,3]
roots=[0,1,2]
parts_of_speech=[nouns,verbs,adv]
parts_of_speech_krasnih=[nouns_krasnih,verbs_krasnih,adv_krasnih]

In [19]:
def vocab(pref,suff,words, root):
    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]<=pref) & (compare(words[i],words[j])[1]<=suff) \
                        & (compare(words[i],words[j])[2]<=root) :
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

In [20]:
def vocab_verb(pref,suff,words, root):
    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]<=pref) & (compare(words[i],words[j])[1]<=suff) \
                        & (compare(words[i],words[j])[2]<=root) &  (particle(words[i])==particle(words[j])):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

In [21]:
def vocab_verb1(pref,suff,words, root):
    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]==pref) & (compare(words[i],words[j])[1]<=suff) \
                        & (compare(words[i],words[j])[2]<=root) &  (particle(words[i])==particle(words[j])):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

In [22]:
def vocab_verb23(pref,suff,words, root):
    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]==pref) & (compare(words[i],words[j])[1]<=suff) \
                        & (compare(words[i],words[j])[2]<=root) &  (particle(words[i])==particle(words[j])):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean 

In [23]:
def vocab_verb3(pref,suff,words, root):
    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]==pref) & (compare(words[i],words[j])[1]==suff) \
                        & (compare(words[i],words[j])[2]<=root) &  (particle(words[i])==particle(words[j])):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

In [24]:
def vocab_verb0(pref,suff,words, root):
    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]==pref) & (compare(words[i],words[j])[1]==suff) \
                        & (compare(words[i],words[j])[2]==root) &  (particle(words[i])==particle(words[j])):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

In [97]:
def vocab_adv0(pref,suff,words, root):
    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]==pref) & (compare(words[i],words[j])[1]==suff) \
                        & (compare(words[i],words[j])[2]==root):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

def vocab_adv23(pref,suff,words, root):
    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]==pref) & (compare(words[i],words[j])[1]<=suff) \
                        & (compare(words[i],words[j])[2]<=root):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean 

def vocab_adv(pref,suff,words, root):
    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]<=pref) & (compare(words[i],words[j])[1]<=suff) \
                        & (compare(words[i],words[j])[2]<=root):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms


In [25]:
def vocab_nouns(pref,suff,words, roots):
#    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]<=pref) & (compare(words[i],words[j])[1]<=suff) \
                        & (compare(words[i],words[j])[2]<=roots) &  (words[i][4]==words[j][4])\
                        & (not((suffix(words[i])==[]) &(suffix(words[j])==['ЧИК']))):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

In [64]:
def vocab_nouns1(pref,suff,words, roots):
#    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]==pref) & (compare(words[i],words[j])[1]<=suff) \
                        & (compare(words[i],words[j])[2]<=roots) &  (words[i][4]==words[j][4])\
                        & (not((suffix(words[i])==[]) &(suffix(words[j])==['ЧИК']))):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

In [26]:
def vocab_nouns0(pref,suff,words, roots):
#    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]==pref) & (compare(words[i],words[j])[1]==suff) \
                        & (compare(words[i],words[j])[2]==roots) &  (words[i][4]==words[j][4])\
                        & (not((suffix(words[i])==[]) &(suffix(words[j])==['ЧИК']))):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

In [27]:
def vocab_nouns3(pref,suff,words, roots):
#    paronyms=[]
    paronyms_clean=[]
    for i in range (len(words)):
        for j in range (i+1,len(words)):
            if words[j][0]=='+':
                break
            else:
                if (compare(words[i],words[j])[0]==pref) & (compare(words[i],words[j])[1]==suff) \
                        & (compare(words[i],words[j])[2]<=roots) &  (words[i][4]==words[j][4])\
                        & (not((suffix(words[i])==[]) &(suffix(words[j])==['ЧИК']))):
#                        paronyms.append([words[i],words[j]])
                        paronyms_clean.append([word(words[i]),word(words[j])])
                else:
                    pass
                
    return paronyms_clean  #paronyms

## Формальный критерий

In [537]:
for i in prefixes:
    for j in suffixes:
        for k in roots:
            crosses=[]
            print("D preff = {}, D suff = {}, D roots = {}".format(i,j,k))
            for a,b in zip(parts_of_speech,parts_of_speech_krasnih):
                crosses.append(cross(vocab(i,j,a,k)[1],vocab_krasn(b)))
            print(pd.DataFrame(np.array(crosses).reshape(1,6),columns=['Nouns_k','Nouns_p','Verbs_k','Verbs_p','Adv_k','Adv_p']))
            print('\n')

D preff = 0, D suff = 0, D roots = 0
    Nouns_k   Nouns_p   Verbs_k   Verbs_p     Adv_k     Adv_p
0  0.008475  0.000821  0.008721  0.000876  0.007165  0.006542


D preff = 0, D suff = 0, D roots = 1
    Nouns_k  Nouns_p   Verbs_k   Verbs_p     Adv_k     Adv_p
0  0.018644  0.00171  0.020349  0.001392  0.008188  0.007156


D preff = 0, D suff = 0, D roots = 2
    Nouns_k   Nouns_p   Verbs_k   Verbs_p     Adv_k     Adv_p
0  0.022034  0.001983  0.023256  0.001508  0.012282  0.010381


D preff = 0, D suff = 1, D roots = 0
    Nouns_k   Nouns_p   Verbs_k   Verbs_p     Adv_k     Adv_p
0  0.315254  0.012424  0.017442  0.001343  0.406346  0.078211


D preff = 0, D suff = 1, D roots = 1
    Nouns_k   Nouns_p  Verbs_k   Verbs_p     Adv_k     Adv_p
0  0.388136  0.012735  0.02907  0.001472  0.501535  0.069681


D preff = 0, D suff = 1, D roots = 2
   Nouns_k   Nouns_p   Verbs_k   Verbs_p     Adv_k     Adv_p
0  0.40339  0.012809  0.031977  0.001522  0.518936  0.067981


D preff = 0, D suff = 2, D r

In [28]:
def f(l):
    n = []
    for i in l:
        if i not in n:
            n.append(i)
    return n


In [29]:
list1=vocab_verb(0,3,verbs,0)
list2=vocab_verb3(1,0,verbs,2)
list3=vocab_verb0(0,0,verbs,1)
pv_clean=f(list1+list2+list3)
del list1,list2,list3

In [989]:
for i in pv_old[0:3000]:
    if i not in pv_clean:
        print(i)

['актерствовать', 'дезактивировать']
['актерствовать', 'заактировать']
['актерствовать', 'реактивировать']
['активизировать', 'дезактивировать']
['активизировать', 'заактировать']
['активизировать', 'реактивировать']
['активировать', 'заактировать']
['актировать', 'дезактивировать']
['актировать', 'реактивировать']
['актуализировать', 'дезактивировать']
['актуализировать', 'заактировать']
['актуализировать', 'реактивировать']
['дезактивировать', 'заактировать']
['заактировать', 'реактивировать']
['арканить', 'заарканивать']
['арканить', 'приарканивать']
['заарканивать', 'приарканить']
['заарканить', 'приарканивать']
['багрить', 'забагроветь']
['багрить', 'побагроветь']
['багроветь', 'обагрить']
['багроветь', 'обагрять']
['забагроветь', 'обагрить']
['забагроветь', 'обагрять']
['обагрить', 'побагроветь']
['обагрять', 'побагроветь']
['баюкать', 'убаюкивать']
['забаюкать', 'убаюкивать']
['побаюкать', 'убаюкивать']
['беднеть', 'победствовать']
['бедовать', 'обеднеть']
['бедовать', 'обеднить

In [988]:
len(pv_old),len(pv_clean)

(38745, 28013)

In [89]:
list1=vocab_verb(0,5,adv,0)  #[1]
list2=vocab_verb(0,2,adv,2) #[1]
list3=vocab_verb0(0,3,adv,1)  #[1]
list4=vocab_verb0(1,2,adv,0) 
list5=vocab_verb23(1,1,adv,1)  #[1]
list12=f(list1+list2)
list34=f(list3+list4)
list125=f(list12+list5)
padv_clean=f(list125+list34)
del list12,list34,list125
del list1,list2,list3,list4,list5

In [96]:
len(padv_clean)

47346

In [1005]:
len(pv_old),len(pv_clean)

(38745, 28013)

In [46]:
list1=vocab_nouns(0,4,nouns,0) 
list2=vocab_nouns3(0,1,nouns,4) 
list3=vocab_nouns(0,2,nouns,2) 
list4=vocab_nouns0(1,0,nouns,0)
list5=vocab_nouns(1,1,nouns,1) 
list12=f(list1+list2)
list34=f(list3+list4)
list125=f(list12+list5)
pn_clean=f(list125+list34)
del list12,list34,list125
del list1,list2,list3,list4,list5

In [1006]:
len(padv_old),len(padv_clean)

(62166, 53861)

In [1007]:
del pn_old, padv_old,pv_old

# Анализ полученного словаря

In [91]:
all_paronyms=pn_clean+pv_clean+padv_clean
len(all_paronyms)

106703

In [92]:
#all_paronyms=pn_clean+pv_clean+padv_clean
cross(all_paronyms,vocab_krasn(krasnih))

[0.9827225130890053, 0.01759088310544221]

In [71]:
#аффиксальный но не критерий внешнего сходства
pv_old=f(vocab_verb23(1,2,verbs,8)+vocab_verb(0,3,verbs,8))
padv_old=vocab_verb1(1,2,adv,8)+vocab_verb(0,3,adv,8)
pn_old=vocab_nouns1(1,2,nouns,8)+vocab_nouns(0,3,nouns,8)
pn_old_set=f(pn_old)
pv_old_set=f(pv_old)
padv_old_set=f(padv_old)
len(pn_old_set+pv_old_set+padv_old_set)

137477

In [94]:
#паронимы, которые не попали в аффиксальные критерий
not_affix=[]
for i in vocab_krasn(krasnih):
    if ([i[0],i[1]]) not in all_paronyms:
        if ([i[1],i[0]]) not in all_paronyms:
            not_affix.append(i)
            
not_affix

[['безоглядный', 'ненаглядный'],
 ['безоглядный', 'неприглядный'],
 ['безоглядный', 'непроглядный'],
 ['горестный', 'горький'],
 ['грех', 'погрешность'],
 ['длинный', 'долгий'],
 ['длительный', 'долгий'],
 ['дымный', 'дымящийся'],
 ['дымовой', 'дымящийся'],
 ['дымчатый', 'дымящийся'],
 ['дымящийся', 'задымленный'],
 ['земельный', 'земной'],
 ['идеализированный', 'идеальный'],
 ['изобразить', 'отразить'],
 ['молодой', 'молодящийся'],
 ['молодящийся', 'моложавый'],
 ['нападение', 'нападки'],
 ['огородить', 'оградить'],
 ['остальной', 'остаточный'],
 ['останки', 'остатки'],
 ['отсталый', 'отстающий'],
 ['рассчитанный', 'расчетливый'],
 ['рассчитанный', 'расчетный'],
 ['седловина', 'сиденье'],
 ['убийственный', 'убойный'],
 ['увлекательный', 'увлекающийся'],
 ['увлекающийся', 'увлеченный'],
 ['чувственный', 'чуткий'],
 ['чувствительный', 'чуткий']]

In [95]:
len(not_affix)

29

In [106]:
#Сравнение с аффиксальным критерием (не вошли в старый, вошли у нас)
aff_verbs=[]
for i in pv_clean:
    if i not in pv_old_set:
        aff_verbs.append(i)

aff_nouns=[]
for i in pn_clean:
    if i not in pn_old_set:
        aff_nouns.append(i)
        
aff_adv=[]
for i in padv_clean:
    if i not in padv_old_set:
        aff_adv.append(i)
        
len(aff_adv+aff_nouns)

463

In [107]:
aff_verbs=[]
for i in pv_old_set:
    if i not in pv_clean:
        aff_verbs.append(i)

aff_gonen=[]
for i in pn_old_set[10000:20000]:
    if i not in pn_clean:
        aff_gonen.append(i)
        
aff_gonea=[]
for i in padv_old_set[-10000:]:
    if i not in padv_clean:
        aff_gonea.append(i)
        

In [118]:
aff_verbs

[['актерствовать', 'дезактивировать'],
 ['актерствовать', 'заактировать'],
 ['актерствовать', 'реактивировать'],
 ['активизировать', 'дезактивировать'],
 ['активизировать', 'заактировать'],
 ['активизировать', 'реактивировать'],
 ['активировать', 'заактировать'],
 ['актировать', 'дезактивировать'],
 ['актировать', 'реактивировать'],
 ['актуализировать', 'дезактивировать'],
 ['актуализировать', 'заактировать'],
 ['актуализировать', 'реактивировать'],
 ['дезактивировать', 'заактировать'],
 ['заактировать', 'реактивировать'],
 ['арканить', 'заарканивать'],
 ['арканить', 'приарканивать'],
 ['заарканивать', 'приарканить'],
 ['заарканить', 'приарканивать'],
 ['багрить', 'забагроветь'],
 ['багрить', 'побагроветь'],
 ['багроветь', 'обагрить'],
 ['багроветь', 'обагрять'],
 ['забагроветь', 'обагрить'],
 ['забагроветь', 'обагрять'],
 ['обагрить', 'побагроветь'],
 ['обагрять', 'побагроветь'],
 ['баюкать', 'убаюкивать'],
 ['забаюкать', 'убаюкивать'],
 ['побаюкать', 'убаюкивать'],
 ['беднеть', 'побе

In [ ]:
TODO
-аффиксальный но не внешнего сходства
-новый словарь засунть в коэффициент?


# Смысловая близость

In [60]:
to_read='coeff.txt'
output=open(to_read, "r", encoding="utf-8")
coeff=list(output.readlines())
output.close()
coeff=[(i.rstrip()).split() for i in coeff]
for i in coeff:
    i[2]=float(i[2])

In [708]:
peresech=[]
k=np.arange(0.2, 0.7,0.05)
for m in k:
    final=[]
    for i in range (len(coeff)):
        if coeff[i][2]<=m:
            final.append([coeff[i][0],coeff[i][1]])
    print("k={}".format(m), cross(final,vocab_krasn(krasnih)))
    peresech.append(cross(final,vocab_krasn(krasnih))[0])
print(k[argmax(peresech)], peresech[arg])

k=0.2 [0.09738219895287958, 0.0046925852107879]
k=0.25 [0.18638743455497384, 0.006203603666399471]
k=0.3 [0.27905759162303667, 0.007306274074378692]
k=0.35 [0.381151832460733, 0.008534583821805393]
k=0.39999999999999997 [0.4827225130890052, 0.009733747175946455]
k=0.44999999999999996 [0.5905759162303665, 0.0111050947575683]
k=0.49999999999999994 [0.6879581151832461, 0.012332006907425483]
k=0.5499999999999999 [0.7591623036649214, 0.013173674455790966]
k=0.5999999999999999 [0.8230366492146597, 0.013966753440601316]
k=0.6499999999999999 [0.8785340314136125, 0.01469390614464478]


NameError: name 'argmax' is not defined

In [709]:
peresech=[]
k=np.arange(0.7, 0.9,0.05)
for m in k:
    final=[]
    for i in range (len(coeff)):
        if coeff[i][2]<=m:
            final.append([coeff[i][0],coeff[i][1]])
    print("k={}".format(m), cross(final,vocab_krasn(krasnih)))
    peresech.append(cross(final,vocab_krasn(krasnih))[0])

k=0.7 [0.9078534031413612, 0.015036681177266342]
k=0.75 [0.9282722513089006, 0.015281321105978074]
k=0.8 [0.9397905759162304, 0.015416988748604312]
k=0.8500000000000001 [0.943979057591623, 0.015466570590354625]
k=0.9000000000000001 [0.9450261780104712, 0.015480407207608985]


In [710]:
peresech=[]
k=np.arange(0.9, 1.0,0.03)
for m in k:
    final=[]
    for i in range (len(coeff)):
        if coeff[i][2]<=m:
            final.append([coeff[i][0],coeff[i][1]])
    print("k={}".format(m), cross(final,vocab_krasn(krasnih)))
    peresech.append(cross(final,vocab_krasn(krasnih))[0])

k=0.9 [0.9450261780104712, 0.015480407207608985]
k=0.93 [0.9450261780104712, 0.015480274442538593]
k=0.9600000000000001 [0.9455497382198953, 0.01548871793552371]
k=0.9900000000000001 [0.9455497382198953, 0.015488585101456235]


In [807]:
final=[]
for i in range (len(coeff)):
    if coeff[i][2]<=0.85:
        final.append([coeff[i][0],coeff[i][1]])

In [947]:
deleted2=[]
for i in range (len(coeff)):
    if coeff[i][2]>=0.98:
        if coeff[i][0]!=coeff[i][1]:
            deleted2.append([coeff[i][0],coeff[i][1]])

In [949]:
cross(deleted2,vocab_krasn(krasnih)) #0.98

[0.0005235602094240838, 8.987956138774042e-05]

In [950]:
len(deleted2)

11126

In [955]:
for i in vocab_krasn(krasnih):
    if i in deleted2:
        print(i)

['законодательный', 'законодательский']


In [979]:
deleted3=[]
for i in range (len(coeff)):
    if coeff[i][2]>=0.85 and coeff[i][2]!=2:
        if coeff[i][0]!=coeff[i][1]:
            deleted3.append([coeff[i][0],coeff[i][1]])
len(deleted3)

28

In [975]:
for i in vocab_krasn(krasnih):
    if i in deleted3:
        print(i)

['вспомнить', 'припомнить']
['выплатить', 'уплатить']
['давний', 'давнишний']
['индийцы', 'индусы']
['крохотный', 'крошечный']
['скепсис', 'скептицизм']
['туристический', 'туристский']
['чудесный', 'чудный']


In [980]:
deleted3

[['заваруха', 'заварушка'],
 ['подруга', 'подружка'],
 ['индийцы', 'индусы'],
 ['книжечка', 'книжка'],
 ['речка', 'речушка'],
 ['речки', 'речушки'],
 ['шкафчики', 'шкафы'],
 ['вдох', 'выдох'],
 ['вдохи', 'выдохи'],
 ['внуки', 'правнуки'],
 ['ободрить', 'подбодрить'],
 ['выдумывать', 'придумывать'],
 ['вывевать', 'провевать'],
 ['вымесить', 'замесить'],
 ['перемешать', 'смешать'],
 ['заснуть', 'уснуть'],
 ['переборонить', 'перебороновать'],
 ['взвевать', 'взвеять'],
 ['вывевать', 'вывеять'],
 ['расставаться', 'расстаться'],
 ['выплатить', 'выплачивать'],
 ['оплатить', 'оплачивать'],
 ['лежавший', 'лежащий'],
 ['давний', 'давнишний'],
 ['крохотный', 'крошечный'],
 ['заставленный', 'уставленный'],
 ['выстроенный', 'построенный'],
 ['ослепительный', 'слепящий']]

In [964]:
to_read='not_in.txt'
output=open(to_read, "r", encoding="utf-8")
not_in_ww=list(output.readlines())
output.close()
not_in_ww=[(i.rstrip()).split() for i in not_in_ww]

In [809]:
with open('smicl.txt',"w") as out:
    for i in final:
        print(i[0], ' ',i[1],file=out)

In [119]:
with open('paronyms.txt',"w") as out:
    for i in all_paronyms:
        print(i[0], ' ',i[1],file=out)

In [125]:
('дедуктивность' in coeff[0]) & ('дедукция' in coeff[0])

True

In [129]:
k=[]
for i in all_paronyms:
    for j in coeff:
        if (i[0] in j) & (i[1] in j):
            k.append(j[2])

KeyboardInterrupt: 

In [121]:
for i in all_paronyms:
    if 

[['дедуктивность', 'дедукция', 0.525934644154],
 ['индуктивности', 'индукторы', 0.365141707196],
 ['индуктивность', 'индуктор', 0.424316471407],
 ['индуктивность', 'индукция', 0.430377338388],
 ['индуктор', 'индукция', 0.379719315603],
 ['продукт', 'продукт', 1.0],
 ['продуктивность', 'продукция', 0.458602359019],
 ['продуктивность', 'продуцент', 0.373490157179],
 ['продукты', 'продукты', 1.0],
 ['продукты', 'продукции', 0.386384115381],
 ['продукты', 'продуценты', 0.396983116919],
 ['продукции', 'продуценты', 0.421153394519],
 ['продукция', 'продуцент', 0.451176851854],
 ['редуктор', 'редукция', 0.130202950753],
 ['абонемент', 'абонент', 0.252760218927],
 ['абонемент', 'абонентка', 0.321506877155],
 ['абонементы', 'абонентки', 0.269836854978],
 ['абонементы', 'абоненты', 0.325712431435],
 ['абонент', 'абонентка', 0.472169059126],
 ['абонентки', 'абоненты', 0.504557886577],
 ['абрикос', 'абрикосик', 0.540874809364],
 ['абрикосики', 'абрикосы', 0.437325708009],
 ['абхаз', 'абхазец', 0.5